In [13]:
#  ***h5_to_h5_converter.ipynb***
#     This program converts existing .h5 files of waveform data into
#     formats that are used as standard inside PyCBC, so that the .h5
#     files can be used within the PyCBC framework.
    
#     The program is dependent upon a properly formatted "metadata.txt"
#     file, which it will convert into code upon running this program, 
#     to collect key parameters about the waveform that is being converted.
    
#     Thorough documentation will eventually be created on all of this, but
#     there have been too many recent changes to make documentation worth
#     it yet.

In [14]:
# import all essential libraries
import numpy as np
import romspline as romSpline
import h5py
import os
from pycbc import pnutils
import lal
import hashlib
import pycbc.types as types
from pycbc.waveform import utils as wfutils

In [15]:
# unit conversion functions

# converts time to units of Mtotal, based on what format it was already saved in...
# ...as defined by variable "timeFormat"
def convert_time(timeFormat, times, total_grav_mass):
    if (timeFormat == "s"):
        for time in times:
            time /= lal.MTSUN.SI
            timeFormat = "Msun"
    if (timeFormat == "Msun"):
        for time in times:
            time /= total_grav_mass
            timeFormat = "Mtotal"
    if (timeFormat == "Mtotal"):
        return times
    else:
        print "Time is in an unrecognized format. Please edit the metadata file and try again."
        sys.exit()
        

# converts strain to units of rhOverM, based on what format it was already saved in...
# ...as defined by variable "strainFormat"
def convert_strain(strainFormat, strainVal1, strainVal2, total_grav_mass):
    if (strainFormat == "rh"):
        for i in range(0, len(strainVal1)):
            strainVal1[i] /= total_grav_mass
        # ONLY if in PlusCross
        if (dataFormat == "PlusCross"):
            for i in range(0, len(strainVal2)):
                strainVal2[i] /= total_grav_mass
        return strainVal1, strainVal2
    elif (strainFormat == "rhOverM"):
        return strainVal1, strainVal2
    else:
        print "Strain is in an unrecognized format. Please edit the metadata file and try again."
        sys.exit()
        

# converts masses to units of Msun, based on what format they were already saved in...
# ...as defined by variable "massFormat"        
def convert_mass(massFormat, grav_mass1, grav_mass2, total_grav_mass):
    if (massFormat == "kg"):
        print "Old mass:"
        print mass1
        grav_mass1 /= lal.MSUN_SI
        grav_mass2 /= lal.MSUN_SI
        massFormat = "Msun"
        print "New mass:"
        print mass1
    elif (massFormat == "mTotal"):
        grav_mass1 *= total_grav_mass
        grav_mass2 *= total_grav_mass
        massFormat = "Msun"
    if (massFormat == "Msun"):
        return grav_mass1, grav_mass2
    else:
        print "Mass1/mass2 is in an unrecognized format. Please edit the metadata file and try again."
        sys.exit()
        
        
# # convert distance to Mpc
# def convert_distance(distFormat, distance):
#     if (distFormat == "Msun"):
#         distance *= lal.MRSUN_SI
#         distFormat = "m"
#     elif (distFormat == "km"):
#         distance *= 1000
#         distFormat = "m"
#     if (distFormat == "m"):
#         distance /= lal.PC_SI
#         distFormat = "pc"
#     if (distFormat == "pc"):
#         distance /= 10e6
#         distFormat = "Mpc"
#     if (distFormat == "Mpc"):
#         return distance
#     else:
#         print "Distance is in an unrecognized format. Please edit the metadata file and try again."
#         sys.exit()

In [16]:
# takes data from metadata.txt and formats it to be used in this conversion script
# finished format per line should look something like the next line when done:
# EOS_name = 'SLy'
def format_metadata(name):
    data = ''
    with open(name, 'r') as file:
        # read a list of lines into data
        line = file.readline()
        while line:
            linea = ''
            lineb = ''
            
            if '= ' not in line:
                if line[0] != '#':
                    line = "# %s" %(line)
            else:
                line = line.replace('\'', '')
                line = line.replace('\"', '')
                line = line.replace('\n','')
                linea, lineb = line.split('= ')
                if '-' in linea:
                    linea = linea.replace('-','_')
                if ',' in lineb:
                    lineb = lineb.replace(' ', '')
                    lineb_new = lineb.split(',')
                    i = 0
                    while i < len(lineb_new):
                        try:
                            float(lineb_new[i])
                        except ValueError:
                            lineb_new[i] = "\'%s\'" %(lineb_new[i])
                        i += 1
                    lineb = ', '.join(lineb_new)
                elif (lineb.lower() == 'nan'):
                    lineb = "\'%s\'" %(lineb)
                else:
                    try:
                        float(lineb)
                    except ValueError:
                        lineb = "\'%s\'" %(lineb)
                line = "%s= %s\n" %(linea,lineb)
                
            data += line
            line = file.readline()
    return data

def write_metadata(metadata):
    data = format_metadata(metadata)
    with open(metadata, 'w') as file:
        file.writelines(data)
    file.close()

def format_h5_file_name(datafile, rad):
    pieces = datafile.split('/')
    res    = pieces[-2]
    radius = rad
        
    return 'Francois_0001_%s_r%s.h5' %(res, radius)

In [17]:
# necessary variables for conversion to put things in correct units
# sometimes these are saved in metadata. 
# they are here only as a backup if not already in metadata
massFormat   = "Msun"
dataFormat   = "MagArg"
timeFormat   = "Msun"
strainFormat = "rhOverM"
delta_t      = 1./(4096*16)

In [18]:
datafiles = []
path ="/home/dwhite/GWPAC/h5_conversions/original_files/Francois/Waveforms/"

for (dirpath, dirnames, filenames) in os.walk(path):
    for file in filenames:
#         # used if file comes in text (.txt, .dat) form rather than .h5 form
#         if (('.txt' in file) or ('.dat' in file)) and ('metadata.txt' not in file):
#             datafiles.append(os.path.join(dirpath, file))
        # used if file comes in standard .h5 form
        if '.h5' in file:
            datafiles.append(os.path.join(dirpath, file))

# go through list and convert each file as necessary into its own .h5
for datafile in datafiles:
    filepath = os.path.dirname(datafile)
    # all Francois' sims are the same (with just different radii/resolutions)
    # so we can use just one metadata file for all compiled sims
    metadata = '/home/dwhite/GWPAC/h5_conversions/original_files/Francois/Waveforms/metadata.txt'

    # format metadata file for our needs and overwrite the old version
    write_metadata(metadata)
    
    # read metadata.txt in as code, so as to be able to call in its stored values
    # as if they were variables
    exec(open(metadata, 'r'))
    
    # create some default variables based on metadata.txt
    # easier to do this here than to change the variable name each other time it's used
    grav_mass1      = max(baryonic_mass1_msol, baryonic_mass2_msol)
    grav_mass2      = min(baryonic_mass1_msol, baryonic_mass2_msol)
    total_grav_mass = grav_mass1 + grav_mass2
    eccentricity    = .0009
    
    h = h5py.File(datafile, 'r')
    h_group = ''
    keys = []
    lmax = 0
    
    for group in h:
        h_group = h[group]
        keys = h_group.keys()
        rad = '0' + str((group.replace('R', '')).replace('.dir',''))
        
        # find the maximum l value for saving in attributes
        # PyCBC will look for this value when reading in a simulation saved as .h5,
        # so it's important to get right
        for key in keys:
            if 'Y_' in key:
                pieces = key.split('_')
                if (int(str(pieces[1])[-1]) > lmax):
                    lmax = int(str(pieces[1])[-1])
    
        # naming convention for the newly converted file (may change per use case)
        h5file = format_h5_file_name(datafile, rad)
        
        # finite or infinite? used for 'NR-techniques' attribute
        waveRad = 'Finite-Radius-Waveform'
        if 'Inf' in rad:
            waveRad = 'Extrapolated-Waveform'

        # start creating the newly formatted .h5, using the naming convention defined above
        with h5py.File('/home/dwhite/GWPAC/h5_conversions/converted_files/Francois/' + h5file,'w') as fd:
            print 'creating file %s from %s...' %(h5file, datafile)
            
            # run mass conversion function for correct PyCBC conventions
            grav_mass1, grav_mass2 = convert_mass(massFormat, grav_mass1, grav_mass2, total_grav_mass)

            # store metadata.txt in our new .h5
            aux = fd.create_group('auxiliary-info')
            mdata = open(metadata, 'r')
            aux.create_dataset('metadata.txt', data=mdata.read())
            mdata.close()

            # check the spin values to give the correct simulation type
            # also used for "NS-spins-meaningful" .h5 attribute
            simtype = 'non-spinning'
            spinval = True
            if ((spin1x != 0) or (spin1y != 0) 
                            or (spin2x != 0) or (spin2y != 0)):
                simtype = 'precessing'
            elif ((spin1z != 0) or (spin2z != 0)):
                simtype = 'aligned-spin'
            else:
                spinval = False
                
            # calculate the value for mean_anomaly (assuming this hasn't been calculated)
            # 0 is default for low eccentricity values (<= .001)
            mean_anom = 0
            if (eccentricity > .001):
                # default value for when it hasn't been properly calculated
                mean_anom = -1

            # create all relevant attributes for converted .h5 file
            
            # default Format 1 attributes
            mchirp, eta = pnutils.mass1_mass2_to_mchirp_eta(grav_mass1, grav_mass2)
            hashtag = hashlib.md5()
            fd.attrs.create('Format', 1)
            fd.attrs.create('type', 'BNS')
            fd.attrs.create('name', h5file)
#             fd.attrs.create('alternative-names', simulation_name)
            fd.attrs.create('NR-group', 'Francois')
#             fd.attrs.create('NR-code', '')
#             fd.attrs.create('modification-date', '')
#             fd.attrs.create('point-of-contact-email', '')
            fd.attrs.create('simulation-type', simtype)
#             fd.attrs.create('INSPIRE-bibtex-keys', '')
            fd.attrs.create('license', 'public')
            fd.attrs.create('Lmax', lmax)
#             fd.attrs.create('files-in-error-series', '')
#             fd.attrs.create('comparable-simulation', '')
            fd.attrs.create('production-run', 1)
            fd.attrs.create('object1', 'NS')
            fd.attrs.create('object2', 'NS')
            fd.attrs.create('mass1', grav_mass1/total_grav_mass)
            fd.attrs.create('mass2', grav_mass2/total_grav_mass)
            fd.attrs.create('eta', eta)
            fd.attrs.create('chirp-mass', mchirp)
            fd.attrs.create('f_lower_at_1MSUN', 133.0)
            fd.attrs.create('spin1x', spin1x)
            fd.attrs.create('spin1y', spin1y)
            fd.attrs.create('spin1z', spin1z)
            fd.attrs.create('spin2x', spin2x)
            fd.attrs.create('spin2y', spin2y)
            fd.attrs.create('spin2z', spin2z)
            # HARDCODING for non-spinning / aligned-spin
            # this, too, could one day be in metadata.txt, if we found it worthy
            fd.attrs.create('LNhatx', 0.0)
            fd.attrs.create('LNhaty', 0.0)
            fd.attrs.create('LNhatz', 1.0)
            fd.attrs.create('nhatx', 1.0)
            fd.attrs.create('nhaty', 0.0)
            fd.attrs.create('nhatz', 0.0)
            fd.attrs.create('Omega', '')
#             fd.attrs.create('eccentricity', '')
#             fd.attrs.create('mean_anomaly', '')
#             fd.attrs.create('NR-techniques', ('Quasi-Equilibrium-ID, ' 
#                             + metric_scheme + ', Psi4-integrated, ' + waveRad))
            
            # hashtag stuff
            # no idea if this is essential, so I'm just leaving it here
            fd.attrs.create('hashtag', hashtag.digest())
            hashtag.update(fd.attrs['type'])
            
            # attributes unique to NS sims
            fd.attrs.create('file-format-version', 2)
            fd.attrs.create('mass1-msol', grav_mass1)
            fd.attrs.create('mass2-msol', grav_mass2)
            fd.attrs.create('baryonic-mass1-msol', baryonic_mass1_msol)
            fd.attrs.create('baryonic-mass2-msol', baryonic_mass2_msol)
            fd.attrs.create('NS-spins-meaningful', spinval)
            fd.attrs.create('EOS-name', '')
            fd.attrs.create('EOS-references', '')
            fd.attrs.create('EOS-remarks', '')
            fd.attrs.create('have-ns-tidal-lambda', False)
#             fd.attrs.create('tidal-lambda1-l2', id_Lambdaell_starA[0])
#             fd.attrs.create('tidal-lambda1-l3', id_Lambdaell_starA[1])
#             fd.attrs.create('tidal-lambda1-l4', id_Lambdaell_starA[2])
#             fd.attrs.create('tidal-lambda2-l2', id_Lambdaell_starB[0])
#             fd.attrs.create('tidal-lambda2-l3', id_Lambdaell_starB[1])
#             fd.attrs.create('tidal-lambda2-l4', id_Lambdaell_starB[2])

            tempkey = h_group[keys[0]][:,0]
            dmax = 0
            maxloc = 0
            strainMag = [0]*(len(tempkey)+10)

            # find the largest amplitude and set its time stamp to t=0; adjust all others
            #    this should be a resonably accurate way of finding the moment of "merger" 
            #    and making it t=0, in keeping with PyCBC conventions

            for key in keys:
                if 'Y_' in key:
                    timeval = h_group[key][:,0]
                    strain1 = h_group[key][:,1]
                    strain2 = h_group[key][:,2]
                    for i in range(0, len(timeval)):
                        strainMag[i] += np.sqrt((strain1[i])**2 + (strain2[i])**2)
                        if strainMag[i] > dmax:
                            maxloc = i
                            dmax = strainMag[i]

            timeAdjust = tempkey[maxloc]

            for key in keys:
                if 'Y_' in key:
                    pieces = key.split('_')
                    if '.dat' in pieces[2]:
                        pieces[2] = pieces[2].replace('.dat', '')
                    # EX: Y_l2_m-1.dat

                    # pull time and strain data from appropriate file(s) and run appropriate conversions
                    # this expects that files used will have three columns, for timestamp (column 1) 
                    # and either plus/cross or magnitude and argument strain data (columns 2 and 3)
                    times      = h_group[key][:,0]
                    strainVal1 = h_group[key][:,1]
                    strainVal2 = h_group[key][:,2]

                    for i in range(0, len(times)):
                        times[i] = (times[i] - timeAdjust)

                    strainVal1, strainVal2 = convert_strain(strainFormat, strainVal1, strainVal2, total_grav_mass)
                    times = convert_time(timeFormat, times, total_grav_mass)

                    times       = np.array(types.TimeSeries(times, delta_t=delta_t))
                    strainVal1  = types.TimeSeries(strainVal1, delta_t=delta_t)
                    strainVal2  = types.TimeSeries(strainVal2, delta_t=delta_t)

                    strainAmp   = []
                    strainPhase = []

                    # run romSpline to convert into reduced order spline, then assign final .h5 values
                    # and write all data to .h5 file
                    # handled independently for Magnitude/Argument vs. Pluss/Cross data, based on 
                    # unique needs for each format
                    if (dataFormat == "MagArg"):
                        strainAmp = np.array(strainVal1)
                        strainPhase = np.array(strainVal2)
                    elif (dataFormat == "PlusCross"):
                        strainAmp = wfutils.amplitude_from_polarizations(strainVal1, strainVal2).data
                        strainPhase = wfutils.phase_from_polarizations(strainVal1, strainVal2).data 
                    else:
                        print "dataFormat is incorrect or is not specified. Edit the metadata file and try again."

                    print '    key = %s' %(key)
                    print '        fitting spline...'

                    try:
                        # when a mode has nothing but zeros for strain, we don't want to add it to the .h5
                        sAmpH = romSpline.ReducedOrderSpline(times, strainAmp, rel=True, verbose=False)
                        sPhaseH = romSpline.ReducedOrderSpline(times, strainPhase, rel=True, verbose=False)

                        grAmp = fd.create_group('amp_' + pieces[1] + '_' + pieces[2])
                        grPhase = fd.create_group('phase_' + pieces[1] + '_' + pieces[2])

                        sAmpH.write(grAmp)
                        sPhaseH.write(grPhase)

                        print '        spline created.'
                    except AssertionError:
                        print '        SPLINE SKIPPED (no strain data).'

            print 'file created.\n'

print 'All done!'

creating file Francois_Lev0_r00159.h5 from /home/dwhite/GWPAC/h5_conversions/original_files/Francois/Waveforms/Lev0/rh_FiniteRadii_CodeUnits.h5...
    key = Y_l2_m-1.dat
        fitting spline...
        spline created.
    key = Y_l2_m-2.dat
        fitting spline...
        spline created.
    key = Y_l2_m0.dat
        fitting spline...
        spline created.
    key = Y_l2_m1.dat
        fitting spline...
        spline created.
    key = Y_l2_m2.dat
        fitting spline...
        spline created.
file created.

creating file Francois_Lev0_r00165.h5 from /home/dwhite/GWPAC/h5_conversions/original_files/Francois/Waveforms/Lev0/rh_FiniteRadii_CodeUnits.h5...
    key = Y_l2_m-1.dat
        fitting spline...
        spline created.
    key = Y_l2_m-2.dat
        fitting spline...
        spline created.
    key = Y_l2_m0.dat
        fitting spline...
        spline created.
    key = Y_l2_m1.dat
        fitting spline...
        spline created.
    key = Y_l2_m2.dat
        fitting s

KeyboardInterrupt: 